In [92]:
# !pip install imblearn

In [100]:
import pandas as pd 
import re
import numpy
pd.set_option('display.max_columns', None)
df = pd.read_csv('data/dataset v1.csv')

<h2>Data preprocessing</h2>

In [101]:
def convert_lease_to_years(lease_str):
    if not isinstance(lease_str, str):
        return np.nan
    years = 0
    months = 0
    # Use regex to find numbers associated with 'year' and 'month'
    year_match = re.search(r'(\d+)\s*year', lease_str)
    month_match = re.search(r'(\d+)\s*month', lease_str)
    if year_match:
        years = int(year_match.group(1))
    if month_match:
        months = int(month_match.group(1))
    # Return the total lease in years, or NaN if no parts were found
    if years == 0 and months == 0:
        return np.nan
    return years + months / 12.0

df['remaining_lease_years'] = df['remaining_lease'].apply(convert_lease_to_years)

In [102]:
floor_map = {'01 TO 03': 2,'04 TO 06': 5,'07 TO 09': 8,'10 TO 12': 11,'13 TO 15': 14,
    '16 TO 18': 17,'19 TO 21': 20,'22 TO 24': 23,'25 TO 27': 26,'28 TO 30': 29,
    '31 TO 33': 32,'34 TO 36': 35,'37 TO 39': 38,'40 TO 42': 41,'43 TO 45': 44,
    '46 TO 48': 47,'49 TO 51': 50,}

df['storey_ordinal'] = df['storey_range'].map(floor_map)

In [103]:
df.drop(columns=['Unnamed: 0','storey_range','street_name','remaining_lease','latitude','longitude','nearest_bus_stop','nearest_pei',
                   'nearest_jc','nearest_kindergarten','nearest_primary_school','nearest_secondary_school','nearest_poly',
                   'nearest_library','nearest_hospital','nearest_mall','nearest_mrt_station','nearest_sports_facility','nearest_hawker_centre'],inplace=True)
X = df.drop(columns=['resale_price'])
y = df['resale_price']

In [104]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

<h2>Sample weights vs using pandas sample to create synthetic data</h2>

In [90]:
# Sample weights
import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

# In your X_train DataFrame
X_train['combined_feature'] = X_train['town'].astype(str) + '_' + \
                            X_train['flat_type'].astype(str) + '_' + \
                            X_train['flat_model'].astype(str)

sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=X_train['combined_feature']  # The imbalanced feature we want to correct for
)

fit_params = {
    'regressor__sample_weight': sample_weights
}
X_train.drop(columns=['combined_feature'],inplace=True)
X_train.drop(columns=['month','town','flat_type','flat_model','block'],inplace=True)
X_test.drop(columns=['month','town','flat_type','flat_model','block'],inplace=True)

model = LinearRegression()
model.fit(X_train, y_train, sample_weight=sample_weights)
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
print(f"\nSample Predictions: {y_pred[:5]}")
print(f"MSE:  {mean_squared_error(y_test, y_pred):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
print(f"MAE:  {mean_absolute_error(y_test, y_pred):.4f}")


Sample Predictions: [512911.11112935 551398.22642726 365799.86221968 747623.09993293
 360417.22653039]
MSE:  7967227787.7531
RMSE: 89259.3289
MAE:  71488.3163


In [105]:
#Using pandas sample to create synthetic data to balance out flat_type,flat_model and town
import pandas as pd

def balance_feature(df, column_name):
    target_count = df[column_name].value_counts().max()
    print(f"Balancing '{column_name}'. Target count for each category will be: {target_count}\n")
    
    balanced_dfs_list = [df]
    
    for category, count in df[column_name].value_counts().items():
        n_needed = target_count - count
        if n_needed > 0:
            subset = df[df[column_name] == category]
            new_samples = subset.sample(n=n_needed, replace=True, random_state=42)
            balanced_dfs_list.append(new_samples)
            
    balanced_df = pd.concat(balanced_dfs_list, ignore_index=True)
    return balanced_df

train_df = pd.concat([X_train, y_train], axis=1)
# --- Balance by 'town' ---
train_df = balance_feature(train_df, 'town')
print("New data distribution for 'town':")
print(train_df['town'].value_counts())

# --- Balance by 'flat_type' ---
train_df = balance_feature(train_df, 'flat_type')
print("New data distribution for 'flat_type':")
print(train_df['flat_type'].value_counts())

# --- Balance by 'flat_model' ---
train_df = balance_feature(train_df, 'flat_model')
print("New data distribution for 'flat_model':")
print(train_df['flat_model'].value_counts())

X_train = train_df.drop(columns=['resale_price'])
y_train = train_df['resale_price']

X_train.drop(columns=['month','town','flat_type','flat_model','block'],inplace=True)
X_test.drop(columns=['month','town','flat_type','flat_model','block'],inplace=True)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
print(f"\nSample Predictions: {y_pred[:5]}")
print(f"MSE:  {mean_squared_error(y_test, y_pred):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
print(f"MAE:  {mean_absolute_error(y_test, y_pred):.4f}")

Balancing 'town'. Target count for each category will be: 11313

New data distribution for 'town':
town
JURONG EAST        11313
KALLANG/WHAMPOA    11313
PUNGGOL            11313
TOA PAYOH          11313
YISHUN             11313
SENGKANG           11313
HOUGANG            11313
BEDOK              11313
TAMPINES           11313
BUKIT BATOK        11313
WOODLANDS          11313
ANG MO KIO         11313
SEMBAWANG          11313
JURONG WEST        11313
BUKIT MERAH        11313
CHOA CHU KANG      11313
PASIR RIS          11313
BUKIT PANJANG      11313
QUEENSTOWN         11313
BISHAN             11313
SERANGOON          11313
GEYLANG            11313
CLEMENTI           11313
CENTRAL AREA       11313
BUKIT TIMAH        11313
MARINE PARADE      11313
Name: count, dtype: int64
Balancing 'flat_type'. Target count for each category will be: 118602

New data distribution for 'flat_type':
flat_type
3 ROOM              118602
4 ROOM              118602
5 ROOM              118602
EXECUTIVE          

<h2>Resetting the training and test data</h2>

In [122]:
df = pd.read_csv('data/dataset v1.csv')

def convert_lease_to_years(lease_str):
    if not isinstance(lease_str, str):
        return np.nan
    years = 0
    months = 0
    # Use regex to find numbers associated with 'year' and 'month'
    year_match = re.search(r'(\d+)\s*year', lease_str)
    month_match = re.search(r'(\d+)\s*month', lease_str)
    if year_match:
        years = int(year_match.group(1))
    if month_match:
        months = int(month_match.group(1))
    # Return the total lease in years, or NaN if no parts were found
    if years == 0 and months == 0:
        return np.nan
    return years + months / 12.0

df['remaining_lease_years'] = df['remaining_lease'].apply(convert_lease_to_years)

floor_map = {'01 TO 03': 2,'04 TO 06': 5,'07 TO 09': 8,'10 TO 12': 11,'13 TO 15': 14,
    '16 TO 18': 17,'19 TO 21': 20,'22 TO 24': 23,'25 TO 27': 26,'28 TO 30': 29,
    '31 TO 33': 32,'34 TO 36': 35,'37 TO 39': 38,'40 TO 42': 41,'43 TO 45': 44,
    '46 TO 48': 47,'49 TO 51': 50,}

df['storey_ordinal'] = df['storey_range'].map(floor_map)

df.drop(columns=['Unnamed: 0','storey_range','street_name','remaining_lease','latitude','longitude','nearest_bus_stop','nearest_pei',
                   'nearest_jc','nearest_kindergarten','nearest_primary_school','nearest_secondary_school','nearest_poly',
                   'nearest_library','nearest_hospital','nearest_mall','nearest_mrt_station','nearest_sports_facility','nearest_hawker_centre'],inplace=True)
X = df.drop(columns=['resale_price'])
y = df['resale_price']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [123]:
# computing sample weights

# In your X_train DataFrame
X_train['combined_feature'] = X_train['town'].astype(str) + '_' + \
                            X_train['flat_type'].astype(str) + '_' + \
                            X_train['flat_model'].astype(str)

sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=X_train['combined_feature']  # The imbalanced feature we want to correct for
)

fit_params = {
    'regressor__sample_weight': sample_weights
}
X_train.drop(columns=['combined_feature'],inplace=True)

<h2>Trying out different types of scalers for X_train and X_test</h2>

In [130]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    StandardScaler, MinMaxScaler, RobustScaler, 
    MaxAbsScaler, QuantileTransformer, PowerTransformer, Normalizer, OneHotEncoder
)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from sklearn.utils.class_weight import compute_sample_weight

categorical_columns = ['town', 'flat_type', 'flat_model']
numerical_columns = [
    'floor_area_sqm', 'lease_commence_date',
    'dist_bus_stop_m', 'dist_pei_m', 'dist_jc_m', 'dist_kindergarten_m',
    'dist_primary_school_m', 'dist_secondary_school_m', 'dist_poly_m',
    'dist_library_m', 'dist_mall_m', 'dist_hospital_m', 'dist_mrt_station_m',
    'dist_sports_facility_m', 'dist_hawker_centre_m',
    'remaining_lease_years','storey_ordinal',]

# Define all scalers to compare
scalers = {
    'StandardScaler': StandardScaler(),
    'MinMaxScaler': MinMaxScaler(),
    'RobustScaler': RobustScaler(),
    'MaxAbsScaler': MaxAbsScaler(),
    'QuantileTransformer': QuantileTransformer(output_distribution='normal'),
    'PowerTransformer': PowerTransformer(method='yeo-johnson'),
    'Normalizer': Normalizer()
}

# Run comparison
print("=== RMSE & MAE Comparison for Different Scalers ===")

for name, scaler in scalers.items():
    # Build ColumnTransformer with current scaler
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', scaler, numerical_columns),
            ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_columns)
        ],
        remainder='drop'
    )

    X_train_preprocessed = preprocessor.fit_transform(X_train)
    X_test_preprocessed = preprocessor.transform(X_test)
    
    model = LinearRegression()
    model.fit(X_train_preprocessed, y_train,sample_weight=sample_weights)
    y_pred = model.predict(X_test_preprocessed)

    print(name)
    print(f"Sample Predictions: {y_pred[:5]}")
    print(f"MSE:  {mean_squared_error(y_test, y_pred):.4f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
    print(f"MAE:  {mean_absolute_error(y_test, y_pred):.4f}")
    print('\n')

=== RMSE & MAE Comparison for Different Scalers ===
StandardScaler
Sample Predictions: [350746.85364092 602076.84831963 319797.66087859 852977.24556406
 394380.98849884]
MSE:  3894070274.5406
RMSE: 62402.4861
MAE:  48858.6420


MinMaxScaler
Sample Predictions: [350746.85364101 602076.84831972 319797.66087869 852977.24556415
 394380.98849894]
MSE:  3894070274.5414
RMSE: 62402.4861
MAE:  48858.6420


RobustScaler
Sample Predictions: [350746.85364093 602076.84831963 319797.6608786  852977.24556406
 394380.98849885]
MSE:  3894070274.5407
RMSE: 62402.4861
MAE:  48858.6420


MaxAbsScaler
Sample Predictions: [350746.85364744 602076.84832617 319797.6608851  852977.24557059
 394380.98850535]
MSE:  3894070274.5973
RMSE: 62402.4861
MAE:  48858.6420


QuantileTransformer
Sample Predictions: [287244.98119676 558089.88309541 276171.50452966 926569.30839539
 482990.55023801]
MSE:  6400347628.9980
RMSE: 80002.1727
MAE:  62856.0792


PowerTransformer
Sample Predictions: [371647.93677326 585607.93137738

<h2>Comparing different types of scalers on y_train and y_test</h2>

In [132]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import (
    StandardScaler, MinMaxScaler, RobustScaler,
    MaxAbsScaler, QuantileTransformer, PowerTransformer, OneHotEncoder
)
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Define your feature columns
categorical_columns = ['town', 'flat_type', 'flat_model']
numerical_columns = [
    'floor_area_sqm', 'lease_commence_date',
    'dist_bus_stop_m', 'dist_pei_m', 'dist_jc_m', 'dist_kindergarten_m',
    'dist_primary_school_m', 'dist_secondary_school_m', 'dist_poly_m',
    'dist_library_m', 'dist_mall_m', 'dist_hospital_m', 'dist_mrt_station_m',
    'dist_sports_facility_m', 'dist_hawker_centre_m',
    'remaining_lease_years','storey_ordinal',]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_columns)
    ],
    remainder='drop'
)
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)


# --- Define all scalers to compare for the target variable ---
target_scalers = {
    'StandardScaler': StandardScaler(),
    'MinMaxScaler': MinMaxScaler(),
    'RobustScaler': RobustScaler(),
    'MaxAbsScaler': MaxAbsScaler(),
    'QuantileTransformer': QuantileTransformer(output_distribution='normal'),
    'PowerTransformer': PowerTransformer(method='yeo-johnson'),
}

print("=== RMSE & MAE Comparison for Different Target Scalers ===")

# --- Loop through each scaler, apply it to y, train, predict, and unscale ---
for name, scaler in target_scalers.items():
    print(f"\n--- Testing Scaler: {name} ---")

    # 1. Reshape y_train to be a 2D array for the scaler
    y_train_reshaped = y_train.values.reshape(-1, 1)
    y_test_reshaped = y_test.values.reshape(-1, 1)

    # 2. Fit the scaler on y_train and transform both y_train and y_test
    y_train_scaled = scaler.fit_transform(y_train_reshaped)
    y_test_scaled = scaler.transform(y_test_reshaped) # Use transform only on test data

    # 3. Train the model on the scaled target variable
    model = LinearRegression()
    model.fit(X_train_preprocessed, y_train_scaled, sample_weight=sample_weights)
    
    # 4. Make predictions. The output will be on the scaled magnitude.
    y_pred_scaled = model.predict(X_test_preprocessed)

    # 5. IMPORTANT: Unscale the predictions back to the original price range
    y_pred_unscaled = scaler.inverse_transform(y_pred_scaled)

    # 6. Calculate metrics using the unscaled predictions and original y_test
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_unscaled))
    mae = mean_absolute_error(y_test, y_pred_unscaled)
    
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE:  {mae:.4f}")

=== RMSE & MAE Comparison for Different Target Scalers ===

--- Testing Scaler: StandardScaler ---
RMSE: 62402.4861
MAE:  48858.6420

--- Testing Scaler: MinMaxScaler ---
RMSE: 62402.4861
MAE:  48858.6420

--- Testing Scaler: RobustScaler ---
RMSE: 62402.4861
MAE:  48858.6420

--- Testing Scaler: MaxAbsScaler ---
RMSE: 62402.4861
MAE:  48858.6420

--- Testing Scaler: QuantileTransformer ---
RMSE: 53045.4391
MAE:  40592.8632

--- Testing Scaler: PowerTransformer ---
RMSE: 53859.3113
MAE:  41205.3897
